# Goal: Create Interactive Map of 2025 NCAA Participants

In [124]:
### Setup and Dependencies

### Import constants from config.py
from config import *

### Dependencies
import os
import geopandas as gpd
import pandas as pd
import numpy as np
import seaborn as sns
import sqlite3  # Assuming SQL connection for database operations

### MATPLOTLIB WITH ACCESORIES
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import matplotlib.image as mpimg
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.legend_handler import HandlerTuple
from matplotlib.legend_handler import HandlerBase
from PIL import Image
## Map visualization
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
from folium.features import CustomIcon


### FILE PATHS
TEMP_FOLDER = '../TEMP/'
DATA_FOLDER = '../data/'

## 2024-25 Full Roster Path
roster_path = DATA_FOLDER + 'roster_2025_current_march_25_v4_ex20250325.csv'

full_roster = pd.read_csv(roster_path) # load roster as dataframe
full_roster.info() # Check to make sure it loaded correctly



### SCHOOL INFO TABLE FOR LOGO PATHS
school_info_path = os.path.join('..', 'data', 'arena_school_info.csv')
school_info_df = pd.read_csv(school_info_path) # Load school info

# Path to logo folder
logo_folder = os.path.join('..', 'images', 'logos')

### SHAPEFILES
# Path to .geojson file with State Boundries
geojson_path = os.path.join('..', 'data', 'vault', 'combined-us-canada.geojson')
# Load the states shapefile
gdf_states = gpd.read_file(geojson_path)

# Path to shapefile with all US counties
shapefile_path = os.path.join('..', 'data', 'vault', 'cb_2018_us_county_500k.shp')
gdf = gpd.read_file(shapefile_path)
# Set the initial CRS (assuming it's in EPSG:4326, but you may need to verify the original CRS)
gdf = gdf.set_crs(epsg=4326)

## CHECK SHAPEFILES FOR COMPATIBILITY
# Set the CRS for both dataframes if it's missing
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)  # Assuming coordinates are in WGS 84 (lat/lon)

if gdf_states.crs is None:
    gdf_states.set_crs(epsg=4326, inplace=True)  # Assuming coordinates are in WGS 84 (lat/lon)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1836 entries, 0 to 1835
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Current Team    1836 non-null   object 
 1   Last_Name       1836 non-null   object 
 2   First_Name      1836 non-null   object 
 3   No              1836 non-null   int64  
 4   Position        1836 non-null   object 
 5   Yr              1836 non-null   object 
 6   Ht              1836 non-null   object 
 7   Wt              1836 non-null   int64  
 8   DOB             1835 non-null   object 
 9   Hometown        1835 non-null   object 
 10  Height_Inches   1836 non-null   int64  
 11  Draft_Year      228 non-null    float64
 12  NHL_Team        228 non-null    object 
 13  D_Round         228 non-null    float64
 14  Last Team       1828 non-null   object 
 15  League          1781 non-null   object 
 16  City            1835 non-null   object 
 17  State_Province  1835 non-null   o

#### Roster Hotfix
- replace missed location abbreviations with full names
    - RUS = Russia
    - IN = S_P 'Indiana' + Country 'USA'
    - UKR = Ukraine
    - JPN = Japan

In [125]:
## Location abbreviation replacement dictionary
loc_replace_dict = {'RUS':'Russia', 'IN':'Indiana', 'UKR':'Ukraine', 'JPN':'Japan'}

# Check the State_Province column and replace the abbreviations with full names
full_roster['State_Province'] = full_roster['State_Province'].replace(loc_replace_dict)
# same with the Country column
full_roster['Country'] = full_roster['Country'].replace(loc_replace_dict)

# If Country = Indiana, change to USA
full_roster['Country'] = full_roster['Country'].replace('Indiana', 'USA')

In [126]:
### Filter Roster to only teams in the NCAA Tournament
print(ncaa_team_list_2025) # Check the list of Tourney teams from the config file

# Filter to only teams in the NCAA Tournament
roster_ncaa = full_roster[full_roster['Current Team'].isin(ncaa_team_list_2025)]

roster_ncaa.rename(columns={'Current Team':'Team'}, inplace=True) # Rename 'Current_Team' to 'Team' for consistency

# Create a new 'Player' column that combines the player's first and last name
roster_ncaa['Player'] = roster_ncaa['First_Name'] + ' ' + roster_ncaa['Last_Name']

roster_ncaa['Player'] = roster_ncaa['Player'].str.strip() # Strip any leading or trailing white space

# roster_ncaa.info() # Check to make sure it loaded correctly

['Michigan State', 'Cornell', 'Boston University', 'Ohio State', 'Western Michigan', 'Minnesota State', 'Minnesota', 'Massachusetts', 'Boston College', 'Bentley', 'Providence', 'Denver', 'Maine', 'Penn State', 'Connecticut', 'Quinnipiac']


C:\Users\jbanc\AppData\Local\Temp\ipykernel_6756\2994776235.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  roster_ncaa.rename(columns={'Current Team':'Team'}, inplace=True) # Rename 'Current_Team' to 'Team' for consistency
C:\Users\jbanc\AppData\Local\Temp\ipykernel_6756\2994776235.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  roster_ncaa['Player'] = roster_ncaa['First_Name'] + ' ' + roster_ncaa['Last_Name']
C:\Users\jbanc\AppData\Local\Temp\ipykernel_6756\2994776235.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

## Add Current Season Stats to Roster

In [127]:
### Load the player_ytd stats table from the database

# connect to the database
conn = sqlite3.connect(recent_clean_db)
# Check the table name in db
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())


# Load the player_ytd stats table from the database
player_ytd = pd.read_sql_query("SELECT * FROM player_stats_ytd", conn)
player_ytd.rename(columns={'Clean_Player':'Player'}, inplace=True) # Rename 'Clean Player' to 'Player' for consistency
player_ytd['Player'] = player_ytd['Player'].str.strip() # Strip leading and trailing whitespace from the 'Player' column
# player_ytd.info() # Check to make sure it loaded correctly

# Close the connection
conn.close()

### Merge the player_ytd stats table with the roster_ncaa table on Player and Team columns
# roster_ncaa_ytd = pd.merge(roster_ncaa, player_ytd, on=['Player', 'Team'], how='left')
roster_ncaa_ytd = pd.merge(roster_ncaa, player_ytd, on='Player', how='left')

roster_ncaa_ytd.rename(columns={'Team_x':'Team'}, inplace=True)# Rename Team_x back to Team
roster_ncaa_ytd.rename(columns={'Team_y':'Team_from_db'}, inplace=True) # Rename Team_y to Team_from_db

# roster_ncaa_ytd.info() # Check to make sure it merged correctly
# roster_ncaa_ytd.head() # Head of the merged table

[('player_stats_ytd',), ('master_roster',), ('advanced_metrics',), ('game_details',), ('goalie_stats',), ('line_chart',), ('linescore',), ('penalty_summary',), ('player_stats',), ('scoring_summary',)]


#### Filter out Skaters who have not appeared in a game this season

In [128]:
## How many players show no games played?
# no_games = roster_ncaa_ytd[roster_ncaa_ytd['Games_Played'].isnull()]
# print(len(no_games))
# print(no_games)



In [129]:
## Drop any players who have not appeared in a game this season
## Print pre filter length
print(f'PreFiltered Roster Length: {len(roster_ncaa_ytd)}')
roster_ncaa_ytd = roster_ncaa_ytd.dropna(subset=['Games_Played'])
print(f'PostFiltered Roster Length: {len(roster_ncaa_ytd)}')

### Value count by Position
# print(roster_ncaa_ytd['Position'].value_counts())

PreFiltered Roster Length: 443
PostFiltered Roster Length: 413


In [130]:
# Group by the location columns and count the number of players in each group
location_summary = roster_ncaa_ytd.groupby(['City', 'State_Province', 'Country']).size().reset_index(name='Player_Count')

# Display the resulting DataFrame
print(location_summary)

               City    State_Province   Country  Player_Count
0        Abbotsford  British Columbia    Canada             1
1          Aberdeen          Scotland  Scotland             1
2          Abington     Massachusetts       USA             1
3           Airdrie           Alberta    Canada             1
4        Albert Lea         Minnesota       USA             1
..              ...               ...       ...           ...
309        Woodbury         Minnesota       USA             3
310  Woodcliff Lake        New Jersey       USA             1
311       Woodhaven          Michigan       USA             1
312       Yaroslavl            Russia    Russia             1
313          Zilina          Slovakia  Slovakia             1

[314 rows x 4 columns]


In [131]:
## Value COunts of the location data (City, State, Country)
# print(len(roster_ncaa_ytd['City'].value_counts()))
# roster_ncaa_ytd['City'].value_counts()
# roster_ncaa_ytd['State_Province'].value_counts()
# roster_ncaa_ytd['Country'].value_counts()

## Get Geocode Locations for all hometowns in the dataset
- Using Google Maps API

In [132]:
# ### GEOCODING USING GOOGLE MAPS API 
# # LCHECK FOR AND LOAD GEOCODED DATA BEFORE RUNNING - THIS COSTS MONEY

# import googlemaps
# import pandas as pd
# import config




# # Initialize the Google Places API client
# gmaps = googlemaps.Client(key=config.g_key)

# def geocode_google_places(row):
#     try:
#         location_str = f"{row['City']}, {row['State_Province']}, {row['Country']}"
#         print(f"Querying location: {location_str}")  # Debugging output
#         geocode_result = gmaps.geocode(location_str)
        
#         # Check the API response
#         print(f"Geocode result: {geocode_result}")  # Debugging output
        
#         # Check if we got a valid result
#         if geocode_result and 'geometry' in geocode_result[0]:
#             location = geocode_result[0]['geometry']['location']
#             return pd.Series([location['lat'], location['lng']])
#         else:
#             return pd.Series([None, None])  # Return None if no valid result
#     except Exception as e:
#         print(f"Error encountered: {e}")  # Debugging output
#         return pd.Series([None, None])  # Handle errors gracefully

# # Apply the geocode function to the data using Google Places API
# location_summary[['Latitude', 'Longitude']] = location_summary.apply(geocode_google_places, axis=1)

# # Filter out rows with missing coordinates if needed
# location_summary_transformed = location_summary.dropna(subset=['Latitude', 'Longitude'])

# # Display the cleaned data with coordinates
# location_summary_transformed.head()

In [133]:
## Save the geocoded data to a CSV file to avoid re-running the API
# location_summary_transformed.to_csv(DATA_FOLDER + '2025_tourney_location_summary_geocoded.csv', index=False)

In [134]:
## Load the geocoded data from the CSV file
location_summary_geocoded = pd.read_csv(DATA_FOLDER + '2025_tourney_location_summary_geocoded.csv')
location_summary_geocoded.head()

,City,State_Province,Country,Player_Count,Latitude,Longitude
0,Abbotsford,British Columbia,Canada,1,49.050438,-122.304470
1,Aberdeen,Scotland,Scotland,1,57.149889,-2.093753
2,Abington,Massachusetts,USA,1,42.104823,-70.945322
3,Airdrie,Alberta,Canada,1,51.292697,-114.013411
4,Albert Lea,Minnesota,USA,1,43.647801,-93.368656


In [135]:
### MERGE THE LATITUDE AND LONGITUDE DATA WITH THE ROSTER DATA
roster_ncaa_ytd_location = pd.merge(roster_ncaa_ytd, location_summary_geocoded, on=['City', 'State_Province', 'Country'], how='left')

# Check the merged data
# print(roster_ncaa_ytd_location.info())
# roster_ncaa_ytd_location.head()



### Talley the Stats by State (Total Games_Played, Goals, Assist, PIM, ect)

In [136]:
# Group by State_Province and add all up player stats to get a state by state summary
state_summary = roster_ncaa_ytd_location.groupby('State_Province').agg({
    'G': 'sum',
    'A': 'sum',
    'Pts': 'sum',
    'plus_minus': 'sum',
    'Sh': 'sum',
    'TOI_sec': 'sum',
    'PIM': 'sum',
    'Games_Played': 'sum'
}).reset_index()

# Calculate the average stats per game for each state
state_summary['GPG'] = state_summary['G'] / state_summary['Games_Played']
state_summary['APG'] = state_summary['A'] / state_summary['Games_Played']
state_summary['PtsPG'] = state_summary['Pts'] / state_summary['Games_Played']
state_summary['plus_minus_PG'] = state_summary['plus_minus'] / state_summary['Games_Played']
state_summary['ShPG'] = state_summary['Sh'] / state_summary['Games_Played']
state_summary['TOI_secPG'] = state_summary['TOI_sec'] / state_summary['Games_Played']
state_summary['PIMPG'] = state_summary['PIM'] / state_summary['Games_Played']

# Calculate the number of players per state
state_summary['Player_Count'] = roster_ncaa_ytd_location.groupby('State_Province').size().values
# Calculate average stats per player for each state
state_summary['Games_PlayedPP'] = state_summary['Games_Played'] / state_summary['Player_Count']
state_summary['GPP'] = state_summary['G'] / state_summary['Player_Count']
state_summary['APP'] = state_summary['A'] / state_summary['Player_Count']
state_summary['PtsPP'] = state_summary['Pts'] / state_summary['Player_Count']
state_summary['plus_minus_PP'] = state_summary['plus_minus'] / state_summary['Player_Count']
state_summary['ShPP'] = state_summary['Sh'] / state_summary['Player_Count']
state_summary['TOI_secPP'] = state_summary['TOI_sec'] / state_summary['Player_Count']
state_summary['PIMPP'] = state_summary['PIM'] / state_summary['Player_Count']


# Display the resulting DataFrame
state_summary.head()

,State_Province,G,A,Pts,plus_minus,Sh,TOI_sec,PIM,Games_Played,GPG,...,PIMPG,Player_Count,Games_PlayedPP,GPP,APP,PtsPP,plus_minus_PP,ShPP,TOI_secPP,PIMPP
0,Alaska,14.0,19.0,33.0,16.0,122.0,59596.0,25.0,68.0,0.205882,...,0.367647,2,34.000000,7.00000,9.500000,16.500000,8.000000,61.000000,29798.00000,12.500000
1,Alberta,127.0,177.0,304.0,156.0,972.0,551595.0,233.0,637.0,0.199372,...,0.365777,20,31.850000,6.35000,8.850000,15.200000,7.800000,48.600000,27579.75000,11.650000
2,Arizona,5.0,6.0,11.0,6.0,35.0,20389.0,8.0,25.0,0.200000,...,0.320000,2,12.500000,2.50000,3.000000,5.500000,3.000000,17.500000,10194.50000,4.000000
3,British Columbia,94.0,216.0,310.0,160.0,989.0,587857.0,317.0,695.0,0.135252,...,0.456115,25,27.800000,3.76000,8.640000,12.400000,6.400000,39.560000,23514.28000,12.680000
4,California,132.0,171.0,303.0,159.0,954.0,495127.0,399.0,614.0,0.214984,...,0.649837,23,26.695652,5.73913,7.434783,13.173913,6.913043,41.478261,21527.26087,17.347826


In [137]:
## Export to csv to check in excel
state_summary.to_csv(TEMP_FOLDER + '2025_tourney_state_summary.csv', index=False)

## Output Breakdown Tables: Hometown and First Name

### Breakdown by Hometown

In [138]:
# # ---- 1) GROUP BY CITY, STATE/PROVINCE, COUNTRY ----
# grouped_city = (
#     roster_ncaa_ytd_location
#     .groupby(["City", "State_Province", "Country"], dropna=False)
#     .agg(
#         Count=("Player", "size"),  # number of players from that city
#         Teams=("Team", lambda x: ", ".join(sorted(set(x))))  # unique teams
#     )
#     .reset_index()
# )

# # Sort by Count descending to see which city has the most players
# grouped_city = grouped_city.sort_values(by="Count", ascending=False)

# # (Optional) Keep the top 10
# top_10_cities = grouped_city.head(10)

# # Print or examine the result
# print(top_10_cities)

### Breakdown by First Name

In [139]:
# # roster_ncaa_ytd_location.info()

# # Strip any leading or trailing whitespace from the 'First_Name' and 'Last_Name' columns
# roster_ncaa_ytd_location['First_Name'] = roster_ncaa_ytd_location['First_Name'].str.strip()
# roster_ncaa_ytd_location['Last_Name'] = roster_ncaa_ytd_location['Last_Name'].str.strip()

# # ---- 2) GROUP BY FIRST NAME ----
# grouped_names = (
#     roster_ncaa_ytd_location
#     .groupby("First_Name", dropna=False)
#     .agg(
#         NumPlayers=("First_Name", "size"),       # total players with that first name
#         NumTeams=("Team", "nunique"),           # how many distinct teams
#         Teams=("Team", lambda x: ", ".join(sorted(set(x)))),
#     )
#     .reset_index()
# )

# # Sort by the number of players descending
# grouped_names = grouped_names.sort_values(by="NumPlayers", ascending=False)

# # (Optional) Take top 10
# top_10_names = grouped_names.head(10)

# # Print or examine the result
# print(top_10_names)

## Create the Map Using Follium

### Circular Offset helper Function

In [140]:
import math

# Function to apply a circular offset to markers with the same location
def add_circular_offset(lat, lon, count, index, radius=0.007):
    """
    Distributes markers in a circular pattern around a central point.
    The radius increases slightly with the number of markers to prevent overlap.
    """
    # Calculate angle in radians (360 degrees divided by number of markers)
    angle = (360 / count) * index
    radians = math.radians(angle)

    # Dynamic adjustment of the radius: the more markers, the larger the radius
    dynamic_radius = radius * (1 + (count / 4))  # Scale the radius based on the number of markers

    # Offset latitude and longitude using circular placement
    lat_offset = lat + (dynamic_radius * math.cos(radians))  # Offset based on cosine
    lon_offset = lon + (dynamic_radius * math.sin(radians))  # Offset based on sine

    return lat_offset, lon_offset


In [141]:
######### RENAME THE DATA TO MERGED_DF
merged_df = roster_ncaa_ytd_location

# Assign unique index per player in each city group
merged_df['city_group_index'] = merged_df.groupby(['City', 'State_Province', 'Country']).cumcount()

# Assign 'Player_Count' per city directly to 'merged_df' using 'transform'
merged_df['Player_Count'] = merged_df.groupby(['City', 'State_Province', 'Country'])['First_Name'].transform('count')

# Set Logo Size (tuple of width and height in pixels)
logo_size = (55, 55)  # Adjust as needed

# Convert all number columns to int
int_columns = ['No', 'Height_Inches', 'Wt', 'Draft_Year', 'D_Round', 
               'G', 'A', 'Pts', 'plus_minus', 'Sh', 'PIM', 'Games_Played']

for col in int_columns:
    merged_df[col] = merged_df[col].astype('Int64')

import math

def create_map_with_team_logos(merged_df, school_info_df, logo_folder, gdf_states, map_center=[45.0, -93.0], zoom_start=4):
    # Initialize the map
    folium_map = folium.Map(location=map_center, zoom_start=zoom_start, tiles='OpenStreetMap', name='Default Map')

    # ---- ADD BASE LAYERS ----
    # Add additional base layers (you can add more as needed)
    # folium.TileLayer('OpenStreetMap', name='Default Map').add_to(folium_map)
    # folium.TileLayer('Stamen Terrain', name='Terrain', attr=".").add_to(folium_map)
    # folium.TileLayer('Stamen Toner', name='Toner', attr=".").add_to(folium_map)
    folium.TileLayer('CartoDB dark_matter', name='Dark Theme', attr=".").add_to(folium_map)
    folium.TileLayer('CartoDB positron', name='Light Theme', attr=".").add_to(folium_map)
    
    ##################### TEST 3/25/25 #####################
    # 1. LOAD/DEFINE THE GEODATAFRAMES
    # -------------------------------
    # Your detailed North American states/provinces GeoDataFrame (already loaded as gdf_states)
    # Make sure it has a column that matches the "State_Province" values in your merged_df.
    # For example, if your gdf_states has a column 'name' for state names:
    gdf_states['State_Province'] = gdf_states['name']

    # Load the global countries dataset (country-level boundaries)
    gdf_global = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

    # Define which countries are part of North America that you already have in detail.
    # Adjust these names as necessary.
    na_countries = ['United States of America', 'Canada']

    # Filter out North American countries from the global dataset to avoid duplicates.
    gdf_global_non_na = gdf_global[~gdf_global['name'].isin(na_countries)].copy()

    # For consistency, create a "State_Province" column in the global dataset as well.
    gdf_global_non_na['State_Province'] = gdf_global_non_na['name']

    # -------------------------------
    # 2. COMBINE THE TWO DATASETS
    # -------------------------------
    # Concatenate the detailed NA data with the non-NA global data.
    gdf_combined = pd.concat([gdf_states, gdf_global_non_na], ignore_index=True)

    # -------------------------------
    # 3. PREPARE THE DATA FOR CHOROPLETH
    # -------------------------------
    # Assume you already have merged_df loaded and processed to count player occurrences.
    territory_counts = merged_df['State_Province'].value_counts()
    territory_counts_df = pd.DataFrame(territory_counts).reset_index()
    territory_counts_df.columns = ['State_Province', 'Player_Count']

    # Define custom bins that work for your distribution.
    custom_bins = [0, 1, 5, 10, 20, 30, 40, 50, 60]  # Ensure the top value exceeds your max count

    # Convert the combined GeoDataFrame to GeoJSON.
    geojson_data = gdf_combined.__geo_interface__

    # -------------------------------
    # 4. ADD THE CHOROPLETH LAYER
    # -------------------------------
    folium.Choropleth(
        geo_data=geojson_data,
        data=territory_counts_df,
        columns=['State_Province', 'Player_Count'],
        key_on='feature.properties.State_Province',  # Note: using the column we set up in both datasets
        fill_color='YlGn',
        fill_opacity=0.5,
        line_opacity=0.2,
        legend_name='Number of Players by Territory',
        bins=custom_bins,
        reset=True,
        name='Shade by Player Count'
    ).add_to(folium_map)


        # -------------------------------
    # 5. ADD THE LABELS LAYER
    # -------------------------------

    # Merge the counts with the combined GeoDataFrame to compute centroids.
    gdf_combined_subset = gdf_combined[['State_Province', 'geometry']]
    territory_counts_gdf = gdf_combined_subset.merge(territory_counts_df,
                                                    on='State_Province',
                                                    how='left')

    # Calculate centroids for label placement.
    territory_counts_gdf['centroid'] = territory_counts_gdf.geometry.centroid

    # Create a FeatureGroup for the labels.
    labels_layer = folium.FeatureGroup(name='Player Count by Territory')

    # Add a label marker for each territory that has a player count.
    for idx, row in territory_counts_gdf.iterrows():
        if pd.notnull(row['Player_Count']):
            lat = row['centroid'].y
            lon = row['centroid'].x
            # Convert player count to an integer to display whole numbers only.
            player_count = int(row['Player_Count'])
            label = folium.Marker(
                location=[lat, lon],
                icon=folium.DivIcon(
                    html=f'''
                        <div class="dynamic-label" style="
                            font-family: Exo 2, sans-serif;
                            font-weight: bold;
                            font-size: 16px;  /* Default size; will be updated dynamically */
                            color: black;
                            text-align: center;
                            padding: 2px;
                        ">
                            {player_count}
                        </div>
                    '''
                )
            )
            labels_layer.add_child(label)

    # Add the labels layer to the map.
    labels_layer.add_to(folium_map)

    # -------------------------------
    # 6. ADD CUSTOM JAVASCRIPT FOR DYNAMIC FONT SIZING
    # -------------------------------

    # This script adjusts the font size of elements with the class 'dynamic-label'
    # based on the map's zoom level. Adjust the multiplier as needed.
    dynamic_font_script = """
    <script>
        function updateLabelSizes() {
            var zoom = map.getZoom();
            // Adjust font size dynamically: for example, zoom level multiplied by 2,
            // constrained between 12px and 30px.
            var newFontSize = Math.max(12, Math.min(30, zoom * 2));  
            var labels = document.getElementsByClassName('dynamic-label');
            for (var i = 0; i < labels.length; i++) {
                labels[i].style.fontSize = newFontSize + 'px';
            }
        }
        map.on('zoomend', updateLabelSizes);
        updateLabelSizes();  // Initial call on load.
    </script>
    """
    folium_map.get_root().html.add_child(folium.Element(dynamic_font_script))
    # # -------------------------------
    # # 5. ADD THE LABELS LAYER
    # # -------------------------------
    # # Merge the counts with the combined GeoDataFrame to compute centroids for labeling.
    # gdf_combined_subset = gdf_combined[['State_Province', 'geometry']]
    # territory_counts_gdf = gdf_combined_subset.merge(territory_counts_df,
    #                                                 on='State_Province',
    #                                                 how='left')

    # # Calculate centroids for label placement.
    # territory_counts_gdf['centroid'] = territory_counts_gdf.geometry.centroid

    # # Create a FeatureGroup for the labels.
    # labels_layer = folium.FeatureGroup(name='Player Count by Territory')

    # # Loop through and add a label for each territory with a player count.
    # for idx, row in territory_counts_gdf.iterrows():
    #     if pd.notnull(row['Player_Count']):
    #         lat = row['centroid'].y
    #         lon = row['centroid'].x
    #         player_count = row['Player_Count']
    #         label = folium.Marker(
    #             location=[lat, lon],
    #             icon=folium.DivIcon(
    #                 html=f'''
    #                     <div style="
    #                         font-family: Exo 2, sans-serif;
    #                         font-weight: bold;
    #                         font-size: 16px;
    #                         color: black;
    #                         text-align: center;
    #                         padding: 2px;
    #                     ">
    #                         {player_count}
    #                     </div>
    #                 '''
    #             )
    #         )
    #         labels_layer.add_child(label)

    # # Add the labels layer to your folium map.
    # labels_layer.add_to(folium_map)
    # # If you need sub-national boundaries globally, you may have to source a different dataset.
    # gdf_territories = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

    # # --- PREPARE THE DATAFRAME WITH PLAYER COUNTS ---
    # # Count players per territory from merged_df (make sure merged_df is already loaded from your CSV)
    # territory_counts = merged_df['State_Province'].value_counts()
    # territory_counts_df = pd.DataFrame(territory_counts).reset_index()
    # territory_counts_df.columns = ['State_Province', 'Player_Count']

    # # --- DEFINE CUSTOM BINS FOR THE CHOROPLETH ---
    # custom_bins = [0, 1, 5, 10, 20, 30, 40, 50, 60]  # Ensure the top value exceeds your max count

    # # --- PREPARE THE GEOJSON DATA ---
    # geojson_data = gdf_territories.__geo_interface__

    # # --- ADD THE CHOROPLETH LAYER TO THE MAP ---
    # folium.Choropleth(
    #     geo_data=geojson_data,
    #     data=territory_counts_df,
    #     columns=['State_Province', 'Player_Count'],
    #     key_on='feature.properties.name',  # Adjust if your geojson uses a different property key
    #     fill_color='YlGn',
    #     fill_opacity=0.5,
    #     line_opacity=0.2,
    #     legend_name='Number of Players by Territory',
    #     bins=custom_bins,
    #     reset=True,
    #     name='Shade by Player Count'
    # ).add_to(folium_map)

    # # --- ADD THE LABELS LAYER ---
    # # Merge your counts with the territories GeoDataFrame so we can position labels by centroid.
    # # (Here we assume the property 'name' in gdf_territories matches the 'State_Province' values.)
    # gdf_subset = gdf_territories[['name', 'geometry']]
    # territory_counts_gdf = gdf_subset.merge(territory_counts_df,
    #                                         left_on='name',
    #                                         right_on='State_Province',
    #                                         how='left')

    # # Compute centroids for label placement. (Note: depending on your CRS, you might need to transform coordinates.)
    # territory_counts_gdf['centroid'] = territory_counts_gdf.geometry.centroid

    # # Create a FeatureGroup to hold the labels.
    # labels_layer = folium.FeatureGroup(name='Player Count by Territory')

    # # Add labels only for territories with a player count.
    # for idx, row in territory_counts_gdf.iterrows():
    #     if pd.notnull(row['Player_Count']):
    #         lat = row['centroid'].y
    #         lon = row['centroid'].x
    #         player_count = row['Player_Count']
    #         label = folium.Marker(
    #             location=[lat, lon],
    #             icon=folium.DivIcon(
    #                 html=f'''
    #                     <div style="
    #                         font-family: Exo 2, sans-serif;
    #                         font-weight: bold;
    #                         font-size: 16px;
    #                         color: black;
    #                         text-align: center;
    #                         padding: 2px;
    #                     ">
    #                         {player_count}
    #                     </div>
    #                 '''
    #             )
    #         )
    #         labels_layer.add_child(label)

    # # Finally, add the labels layer to your folium map.
    # labels_layer.add_to(folium_map)

# ##############################################################
# ##############################################################
#     # ---- ADD CHOROPLETH LAYER ----
#     # Create 'state_counts_df' from 'merged_df'
#     state_counts = merged_df['State_Province'].value_counts()
#     state_counts_df = pd.DataFrame(state_counts).reset_index()
#     state_counts_df.columns = ['State_Province', 'Player_Count']

#     # Create Custom Bins for Choropleth to better control look
#     # Define custom bins to handle the wide distribution
#     # custom_bins = [0, 1, 5, 10, 20, 50, 100, 200, 250] # ORIGINAL FROM PRESEASON MAP Adjust as needed
#     custom_bins = [0, 1, 5, 10, 20, 30, 40, 50, 60] # Last number must be more than top value Adjust as needed

#     # Convert the GeoDataFrame to GeoJSON using __geo_interface__
#     geojson_data = gdf_states.__geo_interface__

#     ### ORIGINAL CODE ###
#     # Add the Choropleth directly to the map with a name for LayerControl
#     folium.Choropleth(
#         geo_data=geojson_data,
#         data=state_counts_df,
#         columns=['State_Province', 'Player_Count'],
#         key_on='feature.properties.name',  # Adjust this if necessary
#         fill_color='YlGn',
#         fill_opacity=0.5,
#         line_opacity=0.2,
#         legend_name='Number of Players by State/Province',
#         bins=custom_bins,  # Apply custom bins
#         reset=True,  # Ensure the choropleth is reset based on new bins
#         name='Shade by Player Count'
#     ).add_to(folium_map)

    
#     # ---- ADD STATE LABELS LAYER ----
#     # Merge 'state_counts_df' with 'gdf_states' to get centroids
#     gdf_states_subset = gdf_states[['name', 'geometry']]  # Adjust 'name' if your GeoDataFrame has a different column name
#     state_counts_gdf = gdf_states_subset.merge(state_counts_df, left_on='name', right_on='State_Province')

#     # Calculate centroids
#     state_counts_gdf['centroid'] = state_counts_gdf.geometry.centroid

#     # Create a FeatureGroup for the labels
#     labels_layer = folium.FeatureGroup(name='Players Count by State')

#     # Add labels to the labels_layer
#     for idx, row in state_counts_gdf.iterrows():
#         # Get centroid coordinates
#         lat = row['centroid'].y
#         lon = row['centroid'].x
#         # Get player count
#         player_count = row['Player_Count']
#         # Create a text label
#         label = folium.Marker(
#             location=[lat, lon],
#             icon=folium.DivIcon(
#                 html=f'''
#                     <div style="
#                         font-family: Exo 2, sans-serif;
#                         font-weight: bold;
#                         font-size: 16px;
#                         color: black;
#                         text-align: center;
                        
#                         padding: 2px;
                        
#                     ">
#                         {player_count}
#                     </div>
#                 '''
#             )
#         )
#         labels_layer.add_child(label)

#     # Add the labels_layer to the map
#     labels_layer.add_to(folium_map)
# ##############################################################
# ##############################################################
    
    # ---- ADD HEATMAP LAYER ----
    # Create heat_data from merged_df
    heat_data = [[row['Latitude'], row['Longitude']] for idx, row in merged_df.iterrows()]

    # Create a FeatureGroup for the heatmap layer
    heatmap_layer = folium.FeatureGroup(name='Heatmap')

    # Add the HeatMap to the FeatureGroup
    HeatMap(heat_data, radius=25, blur=15, max_intensity=20).add_to(heatmap_layer)

    # Add the heatmap layer to the map
    heatmap_layer.add_to(folium_map)

    # ---- MARKER CLUSTER LAYER ----
    cluster_group = folium.FeatureGroup(name='Individual Players', control=True, show=False)
    marker_cluster = MarkerCluster(
        spiderfy_on_max_zoom=True,
        show_coverage_on_hover=False,
        max_cluster_radius=20,
        disableClusteringAtZoom=14,
        animateAddingMarkers=True,
        zoomToBoundsOnClick=True
    ).add_to(cluster_group)

    # Compute the mean latitude and longitude for centering the map
    Latitude = merged_df['Latitude'].mean()
    Longitude = merged_df['Longitude'].mean()

    # Create the map centered on the computed mean Latitude and Longitude
    map_instance = folium.Map(location=[Latitude, Longitude], zoom_start=12)

    # Add the cluster group to the map but initially hidden
    map_instance.add_child(cluster_group)

    # Define a custom script to toggle the visibility of the cluster group on zoom
    map_instance.get_root().html.add_child(folium.Element(f'''
        <script>
            var clusterLayer = {cluster_group.get_name()};
            var map = {map_instance.get_name()};
            map.on('zoomend', function() {{
                if (map.getZoom() >= 14) {{
                    if (!map.hasLayer(clusterLayer)) {{
                        map.addLayer(clusterLayer);
                    }}
                }} else {{
                    if (map.hasLayer(clusterLayer)) {{
                        map.removeLayer(clusterLayer);
                    }}
                }}
            }});
        </script>
    '''))


    # Loop through the merged_df to place markers
    for idx, row in merged_df.iterrows():
        # Retrieve team and logo information
        team_name = row['Team']
        logo_info = school_info_df[school_info_df['Team'] == team_name]['logo_abv'].values

        if len(logo_info) > 0:
            logo_abv = logo_info[0]
            logo_path = os.path.join(logo_folder, f"{logo_abv}.png")

            if os.path.exists(logo_path):
                logo_icon = CustomIcon(logo_path, icon_size=logo_size)

                player_count = row['Player_Count']
                current_index = row['city_group_index']

                # Apply circular offset for overlapping markers
                if player_count > 1:
                    lat_offset, lon_offset = add_circular_offset(
                        row['Latitude'], row['Longitude'], player_count, current_index
                    )
                else:
                    lat_offset, lon_offset = row['Latitude'], row['Longitude']  # No offset if only one player

                # Enhance the tooltip with player information, including hometown
                tooltip_html = f"""
                <div style="font-size: 14px; font-family: Arial;">
                    <strong>{row['First_Name']} {row['Last_Name']} - {row['Team']}</strong><br>
                    {row['Hometown']}<br>
                    {row['Yr']} {row['Position']}<br>
                    {f"<div style='font-size: 12px; color: gray; margin-top: 5px;'>SEASON STATS:<br> {row['Games_Played']} GP, {row['G']} G, {row['A']} A, {row['Pts']} PTS, {row['PIM']} PIM</div>" if pd.notna(row['Games_Played']) else ""}
                </div>
                """


                # Add player marker with the custom logo icon and enhanced tooltip
                folium.Marker(
                    location=[lat_offset, lon_offset],
                    tooltip=folium.Tooltip(tooltip_html),
                    icon=logo_icon
                ).add_to(marker_cluster)

    # Add the marker cluster layer to the map
    cluster_group.add_to(folium_map)

    # ---- ADD LAYER CONTROL ----
            
    folium.LayerControl().add_to(folium_map)

    # Inject custom CSS for styling the LayerControl
    custom_css = """
    <style>
    /* Style for the Layer Control List */
    .leaflet-control-layers-list {
        font-size: 18px;  /* Increase font size */
        line-height: 1.5; /* Ensure adequate spacing between lines */
    }

    /* Style for the checkboxes and radio buttons */
    .leaflet-control-layers input[type="radio"], 
    .leaflet-control-layers input[type="checkbox"] {
        transform: scale(1.5);  /* Scale the size of the checkbox/radio button */
        margin-right: 8px;      /* Add space between the button and label */
    }

    /* Optional: Style the background of the layer control to make it stand out */
    .leaflet-control-layers {
        background-color: white;  /* Ensure the control has a visible background */
        border-radius: 5px;       /* Slight rounding of the control edges */
        padding: 10x;
        box-shadow: 0px 0px 5px rgba(0,0,0,0.3);  /* Add a shadow for better visibility */
    }
    </style>
    """

    # Add the custom CSS to the map's HTML
    folium_map.get_root().html.add_child(folium.Element(custom_css))

    # Return the map after processing all markers
    return folium_map

# Assuming 'gdf_states' is already defined in your code
enhanced_player_map = create_map_with_team_logos(merged_df, school_info_df, logo_folder, gdf_states)

# Save the map to an HTML file for visualization
enhanced_map_file_path = os.path.join('..', 'TEMP', 'MAP', 'player_origin_map_with_stats_v4.html')
enhanced_player_map.save(enhanced_map_file_path)


C:\Users\jbanc\AppData\Local\Temp\ipykernel_6756\4008500292.py:43: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  gdf_global = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
C:\Users\jbanc\AppData\Local\Temp\ipykernel_6756\4008500292.py:104: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  territory_counts_gdf['centroid'] = territory_counts_gdf.geometry.centroid
